In [1]:
!pip install transformers
!pip install torch
#This installs Hugging Face's transformers library, which provides pre-trained language models like GPT.
!pip install tensorflow keras
#Since transformers depends on TensorFlow or PyTorch, we'll need to install them.

In [2]:
# I will now show you how the code and output for a normal propmt based text generation model would look like.

In [3]:
from transformers import pipeline

# pipeline is a high-level function from transformers that makes using pre-trained models easy.
#"text-generation" tells pipeline we want a model specialized in generating text.
#"distilgpt2" is a lightweight version of GPT-2, making it faster.
generator = pipeline("text-generation", model="distilgpt2", framework="pt")

def generate_text(prompt, max_length=50):
    """
    Generates text based on the given prompt.
    
    Args:
    - prompt (str): The input text.
    - max_length (int): Max length of the generated text.
    
    Returns:
    - str: Generated text.
    """
    result = generator("Once upon a time and place,", max_length=100, temperature=0.7, num_return_sequences=1, no_repeat_ngram_size=2)

    return result[0]['generated_text']
# The output of this function is in the form of a list and hence we are returning "result[0]"
#This function takes a prompt and uses the generator model to generate text.
#max_length=50 ensures the response isn’t too long.
#num_return_sequences=1 means we only generate one response at a time.

# Test the function
prompt = "Once upon a time, in a faraway kingdom, a young hero embarked on a journey to find the lost treasure. He..."

print(generate_text(prompt))


C:\Users\rohit\anaconda3\Lib\site-packages\transformers\generation\utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time and place, you see the presence of animals, animals and plants.…

The great dragon-like beasts, who sometimes have large bodies, are as close to the human eye as possible.
A humanoid creature, a humanoid animal, has a very large body, which is about 4 to 5 feet tall. A humanoid being with a light-vision range of 30 feet, weighs about 1,000 pounds. The creature can be seen in a group or in some


In [4]:
# Our goal here is to hash the prompt (basically create a unique identifier for the prompt) and map it to its result creating 
# a sort of lookup that can be used by the function to avoid regeneration of responses for the same prompts.


In [5]:
import hashlib
# used for hashing functions
# We create a cache dictionary to store pairs of prompts and their previously generated responses. 
cache = {}

def generate_text_with_cache(prompt, max_length=50):
    """
    Generates text based on the given prompt with a caching mechanism.
    
    Args:
    - prompt (str): The input text for the model.
    - max_length (int): Max length of the generated text (default is 50).
    
    Returns:
    - str: The generated text.
    """
    
    # We first hash the prompt to create a unique key for caching
    # We use MD5 hash function to create a unique identifier for the prompt
    # This helps ensure that the same prompt always generates the same hash key
    prompt_hash = hashlib.md5(prompt.encode('utf-8')).hexdigest()
    
    # we check if the result is already in the cache
    # If the hashed prompt is found in the cache, we return the cached result
    if prompt_hash in cache:
        print("Fetching from cache...")  # We print a message to indicate that the result is coming from the cache
        return cache[prompt_hash]  
    
    # If the result is not in the cache, generate new text using the model
    # Call the model’s pipeline with the prompt and other settings like max_length
    result = generator(prompt, max_length= max_length + len(prompt.split()), temperature=0.7, num_return_sequences=1, no_repeat_ngram_size=2)
    
    # we adjust the "max_length" of the response based on the prompt length
    # The result is a list, so we take the first generated response
    generated_text = result[0]['generated_text']
    
    # Store the generated text in the cache with the hashed prompt as the key
    # This will allow us to avoid recomputing for the same prompt in the future
    cache[prompt_hash] = generated_text
    
    # Return the generated text
    return generated_text

# Test the function with caching
prompt = "Once upon a time, in a faraway kingdom, a young hero named Alex embarked on a journey to find the lost treasure. On his journey, he encountered a strange forest. As he entered, he saw..."
print(generate_text_with_cache(prompt))  # This will generate new text

print('\n')

print(generate_text_with_cache(prompt))  # This call should fetch from cache, not generate new text


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, in a faraway kingdom, a young hero named Alex embarked on a journey to find the lost treasure. On his journey, he encountered a strange forest. As he entered, he saw...

A young man standing in the dark, one of the most beautiful creatures in existence.
Alex's name was Alex. He was a man who had made life a reality. And he knew the truth


Fetching from cache...
Once upon a time, in a faraway kingdom, a young hero named Alex embarked on a journey to find the lost treasure. On his journey, he encountered a strange forest. As he entered, he saw...

A young man standing in the dark, one of the most beautiful creatures in existence.
Alex's name was Alex. He was a man who had made life a reality. And he knew the truth


In [6]:
# As we can see, the first output was freshly generated by the model but when we ran it with the same prompt again, it was 
# able to figure out that it had come across the same prompt previously and instead of utilizing resources in generating 
# another response for the same prompt, the function pulled the previous response from cache.

In [7]:
# It goes without saying that the output generated by the snippets feels a bit out of place and could be more coherent.
# This can be dealth with by playing around with the model settings (temperature, max length, etc) but since the purpose of
# this project is to display a basic implementation of CAG, the actual content of the output has little to do with it.